<a href="https://colab.research.google.com/github/Saadkhalid913/Sentiment-Analysis-Web-App/blob/master/Sentiment_Model_(Tensorflow_only).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import nltk 
import tensorflow as tf 
import pandas as pd 
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import OneHotEncoder
import keras 
from keras.layers.core import Dense

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# segmenting and parsing data

data = []
with open("./train.txt", "r") as f:
  datapoints = f.read().split("\n")
  for i, datapoint in enumerate(datapoints[ : -1]):
    sentence, emotion = datapoint.split(";")
    data.append([sentence, emotion])

data = np.array(data)




In [ ]:
import re


# defining utility functions 
ps = PorterStemmer()
def replace_not_alphabetical_chars(s: str):
  return re.sub("[^a-zA-Z]", " ", s)
def SplitSentence(s: str):
  s = s.lower()
  s = s.split()
  return s 
def GetStopwords():
  words = stopwords.words("english")
  words.remove("not")
  return words
def GetStem(word: str) -> str:
  global ps 
  return ps.stem(word)


In [ ]:
EnglishStopwords = GetStopwords()

In [ ]:
# pre-processing data 

sentences = []
for datapoint in data[ : ]:
  sentence, emotion = datapoint
  sentence = replace_not_alphabetical_chars(sentence)
  sentence = SplitSentence(sentence)
  # sentence = [GetStem(word) for word in sentence if word not in set(EnglishStopwords)]
  sentence = " ".join(sentence)
  sentences.append(sentence)


In [ ]:
text_encoder = tf.keras.preprocessing.text.Tokenizer(num_words=1500, lower=True)
text_encoder.fit_on_texts(sentences)
# encoded_sentences = tf.keras.preprocessing.sequence.pad_sequences(text_encoder.texts_to_sequences(sentences), maxlen=300)
one_hot_results = text_encoder.texts_to_matrix(sentences, mode="binary")

In [ ]:
# Encoding Y (getting classes)
y = data[ :  ,[-1]]
encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()
x = one_hot_results

In [ ]:
def CreateModel():
  model = tf.keras.models.Sequential()
  model.add(Dense(units = 1500, activation="relu"))
  model.add(Dense(units = 1024, activation="relu"))
  model.add(Dense(units = 512, activation="relu"))
  model.add(keras.layers.Dropout(rate = 0.2))
  model.add(Dense(units = 128, activation="relu"))
  model.add(Dense(units = 32, activation="relu"))
  model.add(Dense(units = 6, activation="sigmoid"))
  model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
model = CreateModel()

In [ ]:
model.fit(x,y, epochs=10, batch_size=64)

Epoch 1/10
250/250 [==============================] - 14s 54ms/step - loss: 0.8988 - accuracy: 0.6595
Epoch 2/10
250/250 [==============================] - 13s 54ms/step - loss: 0.2804 - accuracy: 0.9046
Epoch 3/10
250/250 [==============================] - 13s 54ms/step - loss: 0.1220 - accuracy: 0.9583
Epoch 4/10
250/250 [==============================] - 13s 54ms/step - loss: 0.0655 - accuracy: 0.9778
Epoch 5/10
250/250 [==============================] - 14s 55ms/step - loss: 0.0401 - accuracy: 0.9873
Epoch 6/10
250/250 [==============================] - 14s 55ms/step - loss: 0.0366 - accuracy: 0.9881
Epoch 7/10
250/250 [==============================] - 14s 54ms/step - loss: 0.0325 - accuracy: 0.9891
Epoch 8/10
250/250 [==============================] - 14s 54ms/step - loss: 0.0320 - accuracy: 0.9885
Epoch 9/10
250/250 [==============================] - 13s 54ms/step - loss: 0.0359 - accuracy: 0.9883
Epoch 10/10
250/250 [==============================] - 13s 54ms/step - loss: 0.027

In [ ]:
model.save_weights("TF_ONLY_MODEL")


In [ ]:
def MakePred(s):
  string = [s]
  string = text_encoder.texts_to_matrix(string, mode="binary")
  pred = model.predict(string)
  index = np.argmax(pred)
  print(string)
  print(pred)

  print(encoder.inverse_transform(pred))


# MakePred("i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake")
# MakePred("ive been feeling a little burdened lately wasnt sure why that was")
# MakePred("i feel romantic too")
# MakePred(input())



In [ ]:
# print(encoder.inverse_transform(np.array([[1,0,0,0,0,0]])))
# print(encoder.inverse_transform(np.array([[0,1,0,0,0,0]])))
# print(encoder.inverse_transform(np.array([[0,0,1,0,0,0]])))
# print(encoder.inverse_transform(np.array([[0,0,0,1,0,0]])))
# print(encoder.inverse_transform(np.array([[0,0,0,0,1,0]])))
# print(encoder.inverse_transform(np.array([[0,0,0,0,0,1]])))

emotions = np.array([[['anger']],
                      [['fear']],
                      [['joy']],
                      [['love']],
                      [['sadness']],
                      [['surprise']]])
emotions.reshape(-1)

array(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'], dtype='<U8')